<a href="https://colab.research.google.com/github/shantanu2383/FAMA-FRENCH-5-FACTORS-ST-REVERSAL-LT-REVERSAL-MOMENTUM/blob/main/DATALOAD_%2B_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook imports the raw price data, the raw fundamental data and cleans it in a way, so we can later calculate the FAMA FRENCH FACTORS.**

# 00. Import Pandas

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps
from sqlite3 import connect
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

conn=connect(':memory:')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=99e080904ff88413d1b414f22299fb3b37dd84dc6e8e63ba71459107598165e3
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Mounted at /content/gdrive


In [ ]:
main="/content/gdrive/MyDrive/FIMA 2020 JANUARY/V2"


# 01a. IMPORT REAL TIME PRICE DATA

In [ ]:
filepath="/content/gdrive/MyDrive/FIMA_2023_JANUARY/V2/DATALOAD/RAW_DATA/"
file="QuoteMediaHistoricalPrices.csv"

price_df=pd.read_csv(filepath + file)

print(price_df)



1.   Clean the Data
2.   Parse Relevant Variables



In [ ]:
#SLICE THE DATAFRAME TO ONLY INCLUDE OBSERVATIONS POST 2018


price_df['date']=pd.to_datetime(price_df['date'], format='%Y-%m-%d')
#ONLY KEEP DATA AFTER 2018
price_df['date']=pd.to_datetime(price_df['date'], format='%Y-%m-%d')
price_df=price_df[price_df['date']>="2018-01-01"]

In [ ]:
#Parse Relevant Variables
price_df.to_sql('price_df', conn, if_exists='replace')
query=''' 
          SELECT DISTINCT ticker, date, adj_close, adj_volume
          FROM price_df''
          '''
price_df=pd.read_sql(query, conn)
price_df

,ticker,date,adj_close,adj_volume
0,SSP,2018-01-02 00:00:00,14.843616,277727.0
1,SSP,2018-01-03 00:00:00,14.690589,375003.0
2,SSP,2018-01-04 00:00:00,14.728846,414848.0
3,SSP,2018-01-05 00:00:00,14.652332,327348.0
4,SSP,2018-01-08 00:00:00,14.556690,295746.0
...,...,...,...,...
12489345,ICPT,2022-08-02 00:00:00,12.250000,731107.0
12489346,HTBK,2022-08-02 00:00:00,11.388218,90363.0
12489347,VERS,2022-06-13 00:00:00,32.050000,5428.0
12489348,IHG,2022-08-02 00:00:00,59.079792,115234.0


# 01b. IMPORT ZACKS MASTER DATA

HERE WE IMPORT ZACKS MASTER DATA TO MAP TICKER AND ASSET TYPES TO OUR PRICE DATA

In [ ]:
filepath="/content/gdrive/MyDrive/FIMA_2023_JANUARY/V2/DATALOAD/RAW_DATA/"
file= "ZACKS_MT_2_5c2afb6368dcc3ed48e1a84279323e63.csv"
map=pd.read_csv(filepath + file)
print(map)

In [ ]:

map.to_sql("map", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT ticker, exchange, asset_type, comp_type
        FROM map'''

map=pd.read_sql(query, conn)

In [ ]:
#LINK STOCKS MAP TO MAIN STOCKS DATAFRAME

map.to_sql('map', conn, if_exists='replace')

price_df.to_sql('price_df', conn, if_exists='replace')

query=''' SELECT price_df.*, map.exchange, map.asset_type, map.comp_type
            FROM price_df
            LEFT JOIN map ON price_df.ticker=map.ticker'''
          
price_df_xmap=pd.read_sql(query, conn)

FILTER STOCKS

In [ ]:
price_df_xmap['exchange']=price_df_xmap['exchange'].apply(str)

#keep only relevant exchanges
price_df_xmap=price_df_xmap[(price_df_xmap['exchange']=="NYSE") | (price_df_xmap['exchange']=="AMEX") | (price_df_xmap['exchange']=="NASDAQ")]

#keep only US Based common stocks
price_df_xmap['asset_type']=price_df_xmap['asset_type'].apply(str)
price_df_xmap=price_df_xmap[price_df_xmap['asset_type']=="COM"]

#keep only industrial stocks
price_df_xmap['comp_type']=price_df_xmap['comp_type'].apply(str)
price_df_xmap=price_df_xmap[price_df_xmap['comp_type']=="1.0"]

#01c. CALCULATE DAILY RETURNS

In [ ]:
''' METHODOLOGY FOR CALCULATING RETURNS: 
  SHIFT RETURNS AND TICKER BY 1 TO GET THE PREVIOUS DAY RETURNS. BY SHIFTING THE STOCK BACK 1 WE CAN CHECK WHETHER THE ROWS ARE EQUIVALENT, SO WE DON'T ACCIDENTALLY CALCULATE RETURNS BASED ON
  DIFFERENT STOCKS'''
'''drop rows where ticker does not equal stock checker- this is equivalent to dropping the first stock in each series of stocks as this does not have returns
This method saves a lot of time, as we do not have to loop over the 3 million variables to check if the stock is the same as the next one'''


In [ ]:
price_df=price_df_xmap


price_df=price_df.sort_values(['ticker', 'date'], ascending=True)
#Sort by ticker ascending dates, so stocks are listed consecutively 


price_df['adj_close_prev_day']=price_df['adj_close'].shift(1)
#Get previous day price
price_df['stock checker']=price_df['ticker'].shift(1)



p=['adj_close', 'adj_close_prev_day']
for p in p:
  price_df[p]=pd.to_numeric(price_df[p], errors='coerce')

s=['ticker', 'stock checker']
for s in s:
  price_df[s]=price_df[s].apply(str)

#daily returns=[price at t- price at t-1]/price at t-1
price_df['daily_return']=(price_df['adj_close']-price_df['adj_close_prev_day'])/price_df['adj_close_prev_day']




price_df=price_df[price_df['ticker']==price_df['stock checker']]
#drop stocks where we do not have prior day returns


price_df['daily_return']*=100


In [ ]:
#parse relevant variables
price_df.to_sql('price_df', conn, if_exists='replace')

query= '''
          SELECT DISTINCT ticker, date, adj_close, daily_return, exchange
          FROM price_df
          '''

price_df=pd.read_sql(query, conn)


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


WE NOW HAVE A CLEAN DATAFRAME WITH DAILY PERCENTAGE RETURNS

In [ ]:
cleaned_returns=price_df
cleaned_returns.to_csv(filepath + "cleaned_returns.csv")

# 02A. IMPORT AND CLEAN FUNDAMENTALS DATA

In [ ]:
filepath="/content/gdrive/MyDrive/FIMA_2023_JANUARY/V2/DATALOAD/RAW_DATA/"

file= filepath +"ZACKS_FC_2_e68cc2432a0c63b6f0d313a7f60492e9 (1).csv"
z_fc2=pd.read_csv(file)
print(z_fc2)

In [ ]:
file= filepath +"ZACKS_FR_2_e6829a9bbb77b0cf48af6ea1d3be05ce (1).csv"
z_fr2=pd.read_csv(file)
print(z_fr2)

       m_ticker ticker comp_name     comp_name_2 exchange currency_code  \
0           FML  3346Q       NaN  US BANCORP-OLD      NaN           USD   
1           FML  3346Q       NaN  US BANCORP-OLD      NaN           USD   
2           FML  3346Q       NaN  US BANCORP-OLD      NaN           USD   
3           FML  3346Q       NaN  US BANCORP-OLD      NaN           USD   
4           FML  3346Q       NaN  US BANCORP-OLD      NaN           USD   
...         ...    ...       ...             ...      ...           ...   
208075     BHN#   ZZLL       NaN   ZZLL INFO TEC      NaN           USD   
208076     BHN#   ZZLL       NaN   ZZLL INFO TEC      NaN           USD   
208077     BHN#   ZZLL       NaN   ZZLL INFO TEC      NaN           USD   
208078     BHN#   ZZLL       NaN   ZZLL INFO TEC      NaN           USD   
208079     BHN#   ZZLL       NaN   ZZLL INFO TEC      NaN           USD   

       per_end_date per_type  per_code  per_fisc_year  ...  invty_turn  \
0        2017-12-31      

In [ ]:
file= filepath +"ZACKS_MKTV_2_f958220f90a1b5eb5a48772e5c8517d5.csv"
z_mktv2=pd.read_csv(file)
print(z_mktv2)

       ticker    m_ticker     comp_name  fye per_type per_end_date  \
0           A  A2          AGILENT TECH   10        Q   2022-06-30   
1           A  A2          AGILENT TECH   10        Q   2022-03-31   
2           A  A2          AGILENT TECH   10        Q   2021-12-31   
3           A  A2          AGILENT TECH   10        Q   2021-09-30   
4           A  A2          AGILENT TECH   10        Q   2021-06-30   
...       ...         ...           ...  ...      ...          ...   
342142     ZZ  ZZ            SEALY CORP   11        Q   2018-03-31   
342143     ZZ  ZZ            SEALY CORP   11        Q   2017-12-31   
342144     ZZ  ZZ            SEALY CORP   11        Q   2017-09-30   
342145     ZZ  ZZ            SEALY CORP   11        Q   2017-06-30   
342146     ZZ  ZZ            SEALY CORP   11        Q   2017-03-31   

       active_ticker_flag   mkt_val    ep_val  
0                       Y  35477.55  37000.55  
1                       Y  39747.45  41319.45  
2              

PARSE RELEVANT VARIABLES FROM FUNDAMENTALS DATA

In [ ]:
z_mktv2.to_sql("z_mktv2", conn, if_exists='replace')

query='''
        SELECT DISTINCT ticker, per_end_date, per_type, mkt_val
        FROM z_mktv2
        '''
z_mktv2=pd.read_sql(query, conn)


z_fc2.to_sql("z_fc2", conn, if_exists='replace')

query='''SELECT DISTINCT ticker, exchange, tot_asset, net_income_loss_share_holder, comm_shares_out, per_end_date, per_type, tot_liab
            FROM z_fc2
            '''

z_fc2_clean=pd.read_sql(query, conn)

z_fr2.to_sql("z_fr2", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT ticker, exchange, per_end_date, per_type, book_val_per_share
        FROM z_fr2
        '''
z_fr2_clean=pd.read_sql(query, conn)

In [ ]:
#drop all observations on a yearly basis to only keep the quarterly information

z_fr2_clean=z_fr2_clean[z_fr2_clean['per_type']=="Q"]
z_fc2_clean=z_fc2_clean[z_fc2_clean['per_type']=="Q"]
z_mktv2=z_mktv2[z_mktv2['per_type']=="Q"]

#clean data for join

x=[z_fr2_clean, z_fc2_clean, z_mktv2]

for x in x:
  x['per_end_date']=pd.to_datetime(x['per_end_date'], format='%Y-%m-%d')
  x['ticker']=x['ticker'].apply(str)

z_fr2=z_fr2_clean
z_fc2=z_fc2_clean

<ipython-input-36-06589864a8b9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['per_end_date']=pd.to_datetime(x['per_end_date'], format='%Y-%m-%d')


LINK FUNDAMENTALS DATA TOGETHER 

In [ ]:
z_fr2.to_sql("z_fr2", conn, if_exists='replace')
z_fc2.to_sql("z_fc2", conn, if_exists='replace')
z_mktv2.to_sql("z_mktv2", conn, if_exists='replace')

query=''' 
          SELECT DISTINCT z_fc2.*, z_fr2.book_val_per_share, z_mktv2.mkt_val
          FROM z_fc2
          LEFT JOIN z_fr2 ON z_fc2.ticker=z_fr2.ticker AND z_fr2.per_end_date=z_fc2.per_end_date
          LEFT JOIN z_mktv2 ON z_fc2.ticker=z_mktv2.ticker AND z_fr2.per_end_date=z_mktv2.per_end_date
          
          '''
fundamentals=pd.read_sql(query, conn)

In [ ]:
fundamentals

In [ ]:
#only keep relevant exchanges
fundamentals['exchange']=fundamentals['exchange'].apply(str)

fundamentals=fundamentals[(fundamentals['exchange']=="NYSE") | (fundamentals['exchange']=="AMEX") |(fundamentals['exchange']=="NASDAQ")]



# 02b. CONSTRUCT RELEVANT VARIABLES

In [ ]:

x=['tot_asset', 'net_income_loss_share_holder', 'comm_shares_out', 'book_val_per_share', 'mkt_val' ]
for x in x:
  fundamentals[x]=pd.to_numeric(fundamentals[x], errors='coerce')

#1. market cap= market_val

#2. Book value_original= book_val_per_share * comm_shares_out
#   Book_value_updated=Total assets - Total liabilities

fundamentals['book_value_original']=fundamentals['book_val_per_share'] * fundamentals['comm_shares_out']
fundamentals['book_value_updated']=fundamentals['tot_asset'] - fundamentals['tot_liab']
fundamentals['book_value_delta']=fundamentals['book_value_updated']-fundamentals['book_value_original']
fundamentals['book_value']=fundamentals['book_value_updated']

#3. Operating profitablity= net_income_loss_share_holder/book_value
fundamentals['operating_profitability']=fundamentals['net_income_loss_share_holder']/fundamentals['book_value_updated']

#4.total assets=tot_asset

fundamentals
fundamentals=fundamentals.drop(['book_value_updated', 'book_value_original', 'book_value_delta'], axis=1)

NameError: ignored

In [ ]:
fundamentals.to_csv(filepath + "cleaned_fundamentals_updated.csv")

# 03a. PULL RELEVANT VARIABLES FROM FUNDAMENTALS DATA

WE MATCH THE MARKET CAP OF THE LAST QUARTER TO AVOID LOOK-AHEAD BIAS

In [ ]:
filepath="/content/gdrive/MyDrive/FIMA_2023_JANUARY/V2/DATALOAD/RAW_DATA/"
cleaned_returns=pd.read_csv(filepath + "cleaned_returns.csv")
fundamentals=pd.read_csv(filepath + "cleaned_fundamentals_updated.csv")

In [ ]:
fundamentals

,Unnamed: 0,level_0,index,ticker,exchange,tot_asset,net_income_loss_share_holder,comm_shares_out,per_end_date,per_type,tot_liab,book_val_per_share,mkt_val,book_value,operating_profitability
0,0,26,37,AAC,NYSE,1002.156,20.3805,31.9080,2021-03-31 00:00:00,Q,997.1564,0.1567,315.89,4.9996,4.076426
1,1,27,38,A,NYSE,7872.000,168.0000,322.0000,2017-01-31 00:00:00,Q,3572.0000,13.3540,NaN,4300.0000,0.039070
2,2,28,39,A,NYSE,8016.000,164.0000,321.0000,2017-04-30 00:00:00,Q,3641.0000,13.6293,NaN,4375.0000,0.037486
3,3,29,40,A,NYSE,8261.000,175.0000,322.0000,2017-07-31 00:00:00,Q,3650.0000,14.3199,NaN,4611.0000,0.037953
4,4,30,41,A,NYSE,8426.000,177.0000,322.0000,2017-10-31 00:00:00,Q,3591.0000,15.0155,NaN,4835.0000,0.036608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107192,107192,166872,208055,ZYXI,NASDAQ,90.723,6.1070,38.2514,2021-09-30 00:00:00,Q,27.3400,1.6570,NaN,63.3830,0.096351
107193,107193,166873,208056,ZYXI,NASDAQ,133.274,8.8940,39.7380,2021-12-31 00:00:00,Q,59.3490,1.8603,NaN,73.9250,0.120311
107194,107194,166874,208057,ZYXI,NASDAQ,132.041,1.3770,39.7770,2022-03-31 00:00:00,Q,56.2230,1.9061,NaN,75.8180,0.018162
107195,107195,166875,208058,ZYXI,NASDAQ,119.575,3.3460,38.4040,2022-06-30 00:00:00,Q,50.5660,1.7969,NaN,69.0090,0.048486


In [ ]:
cleaned_returns['date']=pd.to_datetime(cleaned_returns['date'], format='%Y-%m-%d')
cleaned_returns['quarter']=pd.PeriodIndex(cleaned_returns.date, freq='Q')

one_quarter = pd.tseries.offsets.DateOffset(months=3)

cleaned_returns['last_quarter']=cleaned_returns['date'] - one_quarter


In [ ]:
cleaned_returns['last_quarter']=pd.PeriodIndex(cleaned_returns.last_quarter, freq='Q')
fundamentals['per_end_date']=pd.to_datetime(fundamentals['per_end_date'])
fundamentals['quarter']=pd.PeriodIndex(fundamentals.per_end_date, freq='Q')

cleaned_returns['last_quarter']=cleaned_returns['last_quarter'].apply(str)
fundamentals['quarter']=fundamentals['quarter'].apply(str)
cleaned_returns['quarter']=cleaned_returns['quarter'].apply(str)

cleaned_returns

,Unnamed: 0,ticker,date,adj_close,daily_return,exchange,quarter,last_quarter
0,0,A,2018-01-03,66.867163,2.544379,NYSE,2018Q1,2017Q4
1,1,A,2018-01-04,66.365563,-0.750144,NYSE,2018Q1,2017Q4
2,2,A,2018-01-05,67.426640,1.598837,NYSE,2018Q1,2017Q4
3,3,A,2018-01-08,67.571333,0.214592,NYSE,2018Q1,2017Q4
4,4,A,2018-01-09,69.230472,2.455389,NYSE,2018Q1,2017Q4
...,...,...,...,...,...,...,...,...
3967008,3967008,ZYXI,2022-12-21,13.750000,1.401180,NASDAQ,2022Q4,2022Q3
3967009,3967009,ZYXI,2022-12-22,13.600000,-1.090909,NASDAQ,2022Q4,2022Q3
3967010,3967010,ZYXI,2022-12-23,13.810000,1.544118,NASDAQ,2022Q4,2022Q3
3967011,3967011,ZYXI,2022-12-27,13.760000,-0.362056,NASDAQ,2022Q4,2022Q3


In [ ]:
fundamentals.drop('level_0', axis=1, inplace=True)
fundamentals.sort_values(by='mkt_val', ascending=False)

,Unnamed: 0,index,ticker,exchange,tot_asset,net_income_loss_share_holder,comm_shares_out,per_end_date,per_type,tot_liab,book_val_per_share,mkt_val,book_value,operating_profitability,quarter
278,278,489,AAPL,NASDAQ,381191.000,34630.000,16340.8500,2021-12-31,Q,309259.000,4.4020,2897843.25,71932.000,0.481427,2021Q4
279,279,490,AAPL,NASDAQ,350662.000,25010.000,16207.5700,2022-03-31,Q,283263.000,4.1585,2826094.50,67399.000,0.371074,2022Q1
63799,63799,113682,MSFT,NASDAQ,340389.000,18765.000,7500.0000,2021-12-31,Q,180379.000,21.3347,2521346.00,160010.000,0.117274,2021Q4
277,277,488,AAPL,NASDAQ,351002.000,20551.000,16426.7900,2021-09-30,Q,287912.000,3.8407,2321505.00,63090.000,0.325741,2021Q3
63800,63800,113683,MSFT,NASDAQ,344607.000,16728.000,7483.0000,2022-03-31,Q,181683.000,21.7726,2305860.75,162924.000,0.102674,2022Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107192,107192,208055,ZYXI,NASDAQ,90.723,6.107,38.2514,2021-09-30,Q,27.340,1.6570,NaN,63.383,0.096351,2021Q3
107193,107193,208056,ZYXI,NASDAQ,133.274,8.894,39.7380,2021-12-31,Q,59.349,1.8603,NaN,73.925,0.120311,2021Q4
107194,107194,208057,ZYXI,NASDAQ,132.041,1.377,39.7770,2022-03-31,Q,56.223,1.9061,NaN,75.818,0.018162,2022Q1
107195,107195,208058,ZYXI,NASDAQ,119.575,3.346,38.4040,2022-06-30,Q,50.566,1.7969,NaN,69.009,0.048486,2022Q2


In [ ]:
cleaned_returns['ticker']=cleaned_returns['ticker'].apply(str)
fundamentals['ticker']=fundamentals['ticker'].apply(str)

cleaned_returns.to_sql('cleaned_returns', conn, if_exists='replace' )
#fundamentals.drop('level_0', axis=1, inplace=True)
fundamentals.to_sql('fundamentals', conn, if_exists='replace')

query='''SELECT DISTINCT cleaned_returns.*, mkt_val as mkt_cap 
        FROM cleaned_returns
        LEFT JOIN fundamentals
        ON cleaned_returns.last_quarter=fundamentals.quarter AND cleaned_returns.ticker=fundamentals.ticker
        '''

cleaned_returns_mktcap=pd.read_sql(query, conn)


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
cleaned_returns=cleaned_returns_mktcap

In [ ]:
cleaned_returns

,index,Unnamed: 0,ticker,date,adj_close,daily_return,exchange,quarter,last_quarter,mkt_cap
0,0,0,A,2018-01-03 00:00:00,66.867163,2.544379,NYSE,2018Q1,2017Q4,NaN
1,1,1,A,2018-01-04 00:00:00,66.365563,-0.750144,NYSE,2018Q1,2017Q4,NaN
2,2,2,A,2018-01-05 00:00:00,67.426640,1.598837,NYSE,2018Q1,2017Q4,NaN
3,3,3,A,2018-01-08 00:00:00,67.571333,0.214592,NYSE,2018Q1,2017Q4,NaN
4,4,4,A,2018-01-09 00:00:00,69.230472,2.455389,NYSE,2018Q1,2017Q4,NaN
...,...,...,...,...,...,...,...,...,...,...
3971647,3967008,3967008,ZYXI,2022-12-21 00:00:00,13.750000,1.401180,NASDAQ,2022Q4,2022Q3,NaN
3971648,3967009,3967009,ZYXI,2022-12-22 00:00:00,13.600000,-1.090909,NASDAQ,2022Q4,2022Q3,NaN
3971649,3967010,3967010,ZYXI,2022-12-23 00:00:00,13.810000,1.544118,NASDAQ,2022Q4,2022Q3,NaN
3971650,3967011,3967011,ZYXI,2022-12-27 00:00:00,13.760000,-0.362056,NASDAQ,2022Q4,2022Q3,NaN


WE PULL BOOK VALUE FROM THE END OF EACH YEAR TO CALCULATE B/M RATIO

In [ ]:
#PARSE RELEVANT VARIABLES

fundamentals.to_sql('fundamentals', conn, if_exists='replace')

query='''
      SELECT DISTINCT ticker, per_end_date as date, book_value
      FROM fundamentals
      '''

fundamentals_be=pd.read_sql(query, conn)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:

fundamentals_be['date']=pd.to_datetime(fundamentals_be['date'], format='%Y-%m-%d')
fundamentals_be['quarter']=fundamentals_be['date'].dt.quarter
fundamentals_be['month']=fundamentals_be['date'].dt.month
fundamentals_be=fundamentals_be[fundamentals_be['month']==12]
fundamentals_be
#assert fundamentals_be['month']==12

,ticker,date,book_value,quarter,month
28,AA,2017-12-31,6798.000,4,12
33,AA,2018-12-31,7588.000,4,12
37,AA,2019-12-31,5886.000,4,12
41,AA,2020-12-31,5016.000,4,12
45,AA,2021-12-31,6284.000,4,12
...,...,...,...,...,...
107177,ZYXI,2017-12-31,4.902,4,12
107181,ZYXI,2018-12-31,9.291,4,12
107185,ZYXI,2019-12-31,19.653,4,12
107189,ZYXI,2020-12-31,56.855,4,12


We add a reference_date for matching purposes. The reference date is June of next year.
This means for a stock in year t, the relevant book value observation will be book value of t-1

In [ ]:
#add reference date for matching
fundamentals_be['year']=fundamentals_be['date'].dt.year

fundamentals_be['year']=pd.to_numeric(fundamentals_be['year'], errors='coerce')

#ADD REFERENCE DATE FOR MATCHING (6/1 of next calendar year)
fundamentals_be['reference_date']=fundamentals_be['year']+1
fundamentals_be['reference_date']=fundamentals_be['reference_date'].apply(str)
fundamentals_be['reference_date'] = fundamentals_be['reference_date'].apply(lambda x: x + "-06-01")
fundamentals_be['reference_date']=pd.to_datetime(fundamentals_be['reference_date'])
fundamentals_be.drop('year', inplace=True, axis=1)

<ipython-input-13-0ee666016aaf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamentals_be['year']=fundamentals_be['date'].dt.year
<ipython-input-13-0ee666016aaf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamentals_be['year']=pd.to_numeric(fundamentals_be['year'], errors='coerce')
<ipython-input-13-0ee666016aaf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

# 03b. CONSTRUCT THE STOCKS DATA

In [ ]:
cleaned_returns

cleaned_returns.to_sql('cleaned_returns', conn, if_exists='replace')

query=''' 
        SELECT DISTINCT ticker, exchange, date, daily_return as ret, quarter, mkt_cap
        FROM cleaned_returns
        '''

main_stocks_df=pd.read_sql(query, conn)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


ADD REFERENCE DATE FOR MATCHING
IF BEFORE JUNE, REFERENCE DATE IS PREVIOUS YEAR, IF AFTER JUNE CURRENT YEAR

In [ ]:
#here we edit the code again to avoid having to loop over all the variables



main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'])

#create two slices of the dataframe, add reference date seperately and then append back together

main_stocks_df['reference_date']=""

main_stocks_df_beforeJune= main_stocks_df[main_stocks_df['date'].dt.month<6]
main_stocks_df_afterJune =main_stocks_df[main_stocks_df['date'].dt.month>=6]

main_stocks_df_beforeJune['year']=main_stocks_df_beforeJune['date'].dt.year - 1
main_stocks_df_afterJune['year']=main_stocks_df_afterJune['date'].dt.year 

main_stocks_df_beforeJune['year']=(main_stocks_df_beforeJune['year']).apply(str)
main_stocks_df_afterJune['year']=(main_stocks_df_afterJune['year']).apply(str)

main_stocks_df_afterJune['reference_date']= main_stocks_df_afterJune['year'] + "-06-01"
main_stocks_df_beforeJune['reference_date']= main_stocks_df_beforeJune['year'] + "-06-01"

main_stocks_df_refdate=main_stocks_df_beforeJune.append(main_stocks_df_afterJune)

#check datafranes
main_stocks_df_beforeJune

main_stocks_df=main_stocks_df_refdate.drop('year', axis=1)

<ipython-input-15-75ef5a079191>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_stocks_df_beforeJune['year']=main_stocks_df_beforeJune['date'].dt.year - 1
<ipython-input-15-75ef5a079191>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_stocks_df_afterJune['year']=main_stocks_df_afterJune['date'].dt.year
<ipython-input-15-75ef5a079191>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

# 03c. MERGE THE BOOK MARKET VALUES INTO THE STOCKS DATA

In [ ]:
fundamentals_be['reference_date']=pd.to_datetime(fundamentals_be['reference_date'], format='%Y-%m-%d')
main_stocks_df['reference_date']=pd.to_datetime(main_stocks_df['reference_date'], format= '%Y-%m-%d')

fundamentals_be.to_sql('fundamentals_be', conn, if_exists='replace')
main_stocks_df.to_sql('main_stocks_df', conn, if_exists='replace')

query='''
        SELECT DISTINCT main_stocks_df.*, fundamentals_be.book_value
        FROM main_stocks_df
        LEFT JOIN fundamentals_be ON main_stocks_df.ticker=fundamentals_be.ticker AND main_stocks_df.reference_date=fundamentals_be.reference_date
        '''

main_stocks_df_be=pd.read_sql(query, conn)


In [ ]:
main_stocks_df=main_stocks_df_be

# 03d. ADD MARKET EQUITY FROM END OF YEAR COLUMN TO CALCULATE BM RATIO

In [ ]:
#Add market equity data from end of year-1 (used for BM ratio of reference date (year y))


main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'])

stocks_me=main_stocks_df[main_stocks_df['date'].dt.month==12]




#we are effectively trying to shift the market cap back by 1, so we can calculate BE/ME as in FF
#Who calculate it as the book equity for the last fiscal year in t-1 divided by ME for December of t-1

stocks_me.to_sql('stocks_me', conn, if_exists='replace')

query=''' SELECT DISTINCT stocks_me.ticker, quarter, mkt_cap as mkt_equity
            FROM stocks_me
            '''

stocks_me=pd.read_sql(query, conn)
                                  

In [ ]:
#extract year

stocks_me['quarter']=stocks_me['quarter'].apply(str)
stocks_me[['year', 'q']]=stocks_me['quarter'].str.split('Q', expand=True)
stocks_me['year']=pd.to_numeric(stocks_me['year'])
stocks_me['reference_date']=stocks_me['year'] + 1
stocks_me['reference_date']=stocks_me['reference_date'].apply(str)
stocks_me['reference_date']+= "-06-01"
stocks_me['reference_date']=pd.to_datetime(stocks_me['reference_date'], format="%Y-%m-%d")

In [ ]:
#match back into main stocks dataframe
main_stocks_df['reference_date']=pd.to_datetime(main_stocks_df['reference_date'], format="%Y-%m-%d")
stocks_me['quarter']=stocks_me['quarter'].apply(str)

main_stocks_df.to_sql('main_stocks_df', conn, if_exists='replace')
stocks_me.to_sql('stocks_me', conn, if_exists='replace')

query='''
        SELECT DISTINCT main_stocks_df.*, stocks_me.mkt_equity
        FROM main_stocks_df
        LEFT JOIN stocks_me ON main_stocks_df.ticker=stocks_me.ticker AND main_stocks_df.reference_date=stocks_me.reference_date
          '''
main_stocks_df_me=pd.read_sql(query, conn)

CACLULATE BM RATIO:  BE/ME

In [ ]:
main_stocks_df=main_stocks_df_me
main_stocks_df['bm_ratio']=main_stocks_df['book_value']/main_stocks_df['mkt_equity']

In [ ]:
main_stocks_df

,level_0,index,ticker,exchange,date,ret,quarter,mkt_cap,reference_date,book_value,mkt_equity,bm_ratio
0,0,0,A,NYSE,2018-01-03 00:00:00,2.544379,2018Q1,NaN,2017-06-01 00:00:00,NaN,NaN,NaN
1,1,1,A,NYSE,2018-01-04 00:00:00,-0.750144,2018Q1,NaN,2017-06-01 00:00:00,NaN,NaN,NaN
2,2,2,A,NYSE,2018-01-05 00:00:00,1.598837,2018Q1,NaN,2017-06-01 00:00:00,NaN,NaN,NaN
3,3,3,A,NYSE,2018-01-08 00:00:00,0.214592,2018Q1,NaN,2017-06-01 00:00:00,NaN,NaN,NaN
4,4,4,A,NYSE,2018-01-09 00:00:00,2.455389,2018Q1,NaN,2017-06-01 00:00:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3975388,3971647,3971647,ZYXI,NASDAQ,2022-12-21 00:00:00,1.401180,2022Q4,NaN,2022-06-01 00:00:00,73.925,NaN,NaN
3975389,3971648,3971648,ZYXI,NASDAQ,2022-12-22 00:00:00,-1.090909,2022Q4,NaN,2022-06-01 00:00:00,73.925,NaN,NaN
3975390,3971649,3971649,ZYXI,NASDAQ,2022-12-23 00:00:00,1.544118,2022Q4,NaN,2022-06-01 00:00:00,73.925,NaN,NaN
3975391,3971650,3971650,ZYXI,NASDAQ,2022-12-27 00:00:00,-0.362056,2022Q4,NaN,2022-06-01 00:00:00,73.925,NaN,NaN


In [ ]:
main_stocks_df

,ticker,exchange,date,ret,quarter,mkt_cap,reference_date,book_value,mkt_equity,bm_ratio
0,A,NYSE,2018-01-03,2.544379,2018Q1,NaN,2017-06-01,NaN,NaN,NaN
1,A,NYSE,2018-01-04,-0.750144,2018Q1,NaN,2017-06-01,NaN,NaN,NaN
2,A,NYSE,2018-01-05,1.598837,2018Q1,NaN,2017-06-01,NaN,NaN,NaN
3,A,NYSE,2018-01-08,0.214592,2018Q1,NaN,2017-06-01,NaN,NaN,NaN
4,A,NYSE,2018-01-09,2.455389,2018Q1,NaN,2017-06-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3975388,ZYXI,NASDAQ,2022-12-21,1.401180,2022Q4,NaN,2022-06-01,73.925,NaN,NaN
3975389,ZYXI,NASDAQ,2022-12-22,-1.090909,2022Q4,NaN,2022-06-01,73.925,NaN,NaN
3975390,ZYXI,NASDAQ,2022-12-23,1.544118,2022Q4,NaN,2022-06-01,73.925,NaN,NaN
3975391,ZYXI,NASDAQ,2022-12-27,-0.362056,2022Q4,NaN,2022-06-01,73.925,NaN,NaN


In [ ]:
main_stocks_df.to_csv(filepath + "pre sort: cleaned returns with mkt cap + bm")

#04A. SIZE SORTS

SORT STOCKS INTO BIG AND SMALL STOCKS BASED ON NYSE MEDIAN BREAKPOINT

In [ ]:
filepath="/content/gdrive/MyDrive/FIMA_2023_JANUARY/V2/DATALOAD/RAW_DATA/"

main_stocks_df=pd.read_csv(filepath + "pre sort: cleaned returns with mkt cap + bm")

In [ ]:
main_stocks_df.replace([np.inf, -np.inf], inplace=True)
main_stocks_df=main_stocks_df.dropna()

#extract quarter number from quarter column--> This is important as previously we were working with monthly data, but now market cap is on a monthly basis

main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'], format='%Y-%m-%d')
main_stocks_df['quarter']=main_stocks_df['date'].dt.quarter

FILTER FOR NYSE STOCKS ONLY TO GET BREAK POINTS

In [ ]:

#filter for NYSE stocks in June
size_breakpoints=main_stocks_df[(main_stocks_df['date'].dt.month==6) & (main_stocks_df['exchange']=="NYSE")]

size_breakpoints.to_sql("size_breakpoints", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT ticker, reference_date, mkt_cap
        FROM size_breakpoints
        '''
size_breakpoints=pd.read_sql(query, conn)
size_breakpoints['reference_date']=pd.to_datetime(size_breakpoints['reference_date'])
size_breakpoints

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,ticker,reference_date,mkt_cap
0,AA,2019-06-01,5224.02
1,AA,2020-06-01,1145.26
2,AA,2021-06-01,6053.58
3,AA,2022-06-01,16605.91
4,AAC,2022-06-01,1226.25
...,...,...,...
3493,ZVIA,2022-06-01,305.88
3494,ZWS,2019-06-01,2633.94
3495,ZWS,2020-06-01,2761.41
3496,ZWS,2021-06-01,5637.67


In [ ]:
g=size_breakpoints.groupby(['reference_date'])
size_sorts=main_stocks_df[(main_stocks_df['date'].dt.month==6)]



size_sorts['size_median']=""

for i in size_sorts.index:
  size_sorts['size_median'][i]=g.get_group(g.get_group(size_sorts['reference_date'][i]).reset_index()['reference_date'][0])['mkt_cap'].quantile(0.5)
  


<ipython-input-5-07c4b0049705>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_sorts['size_median']=""
<ipython-input-5-07c4b0049705>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_sorts['size_median'][i]=g.get_group(g.get_group(size_sorts['reference_date'][i]).reset_index()['reference_date'][0])['mkt_cap'].quantile(0.5)


ASSIGN TO RELEVANT SIZE PORTFOLIO

In [ ]:
size_sorts['size_portfolio']=""

for i in size_sorts.index:
  if size_sorts['mkt_cap'][i] > size_sorts['size_median'][i]:
    size_sorts['size_portfolio'][i]="B"
  else:
    size_sorts['size_portfolio'][i]="S"

<ipython-input-6-38a26deff3da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_sorts['size_portfolio']=""
<ipython-input-6-38a26deff3da>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_sorts['size_portfolio'][i]="B"
<ipython-input-6-38a26deff3da>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_sorts['size_portfolio'][i]="S"


MERGE BACK INTO MAIN STOCKS DF

In [ ]:
#merge size sorts data back into stocks data

main_stocks_df.to_sql("main_stocks_df", conn, if_exists='replace')
size_sorts.to_sql("size_sorts", conn, if_exists='replace')

query='''
        SELECT DISTINCT main_stocks_df.*, size_sorts.size_median, size_sorts.size_portfolio
        FROM main_stocks_df
        LEFT JOIN size_sorts
        ON main_stocks_df.reference_date=size_sorts.reference_date AND main_stocks_df.ticker=size_sorts.ticker
        '''

main_stocks_df_size=pd.read_sql(query, conn)
main_stocks_df_size.drop('index', inplace=True, axis=1)
main_stocks_df_size

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,Unnamed: 0,ticker,exchange,date,ret,quarter,mkt_cap,reference_date,book_value,mkt_equity,bm_ratio,size_median,size_portfolio
0,1610,AA,NYSE,2019-06-03 00:00:00,1.510146,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B
1,1611,AA,NYSE,2019-06-04 00:00:00,1.766620,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B
2,1612,AA,NYSE,2019-06-05 00:00:00,-2.375514,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B
3,1613,AA,NYSE,2019-06-06 00:00:00,0.935891,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B
4,1614,AA,NYSE,2019-06-07 00:00:00,-0.602689,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213878,3974412,ZYNE,NASDAQ,2022-12-21 00:00:00,1.929825,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S
2213879,3974413,ZYNE,NASDAQ,2022-12-22 00:00:00,-7.056799,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S
2213880,3974414,ZYNE,NASDAQ,2022-12-23 00:00:00,-0.018519,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S
2213881,3974415,ZYNE,NASDAQ,2022-12-27 00:00:00,-7.390257,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S


In [ ]:
main_stocks_df=main_stocks_df_size

# 04B. VALUE SORTS

In [ ]:
main_stocks_df.replace([np.inf, -np.inf], inplace=True)
main_stocks_df=main_stocks_df.dropna()

In [ ]:
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'], format="%Y-%m-%d")

#filter for NYSE stocks in June
value_breakpoints=main_stocks_df[(main_stocks_df['date'].dt.month==6) & (main_stocks_df['exchange']=="NYSE")]

value_breakpoints.to_sql("value_breakpoints", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT ticker, reference_date, bm_ratio
        FROM value_breakpoints
        '''

value_breakpoints=pd.read_sql(query, conn)
value_breakpoints['reference_date']=pd.to_datetime(value_breakpoints['reference_date'])

<ipython-input-10-bba78386c6d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'], format="%Y-%m-%d")
/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
g=value_breakpoints.groupby(['reference_date'])
value_sorts=main_stocks_df[(main_stocks_df['date'].dt.month==6)]


value_sorts['value_q30']=""
value_sorts['value_q70']=""


for i in value_sorts.index:
  value_sorts['value_q30'][i]=g.get_group(g.get_group(value_sorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
  value_sorts['value_q70'][i]=g.get_group(g.get_group(value_sorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)

<ipython-input-11-1f6c071d10ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_sorts['value_q30']=""
<ipython-input-11-1f6c071d10ed>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_sorts['value_q70']=""
<ipython-input-11-1f6c071d10ed>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_sorts['value_q30'][

In [ ]:
#sort values into high, medium, and low portfolios

value_sorts['value_portfolio']=""

for i in value_sorts.index:
  if value_sorts['bm_ratio'][i]>value_sorts['value_q70'][i]:
    value_sorts['value_portfolio'][i]="H"
  elif value_sorts['value_q30'][i]<value_sorts['bm_ratio'][i] <= value_sorts['value_q70'][i]:
    value_sorts['value_portfolio'][i]="M"
  else:
    value_sorts['value_portfolio'][i]="L"

<ipython-input-12-99713e1c3ece>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_sorts['value_portfolio']=""
<ipython-input-12-99713e1c3ece>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_sorts['value_portfolio'][i]="H"
<ipython-input-12-99713e1c3ece>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_sorts['value_portfolio'][i]="L"
<ipython-input-12-99713e1c3e

In [ ]:
#merge back into main df

main_stocks_df.to_sql("main_stocks_df", conn, if_exists='replace')
value_sorts.to_sql("value_sorts", conn, if_exists='replace')

query='''
        SELECT DISTINCT main_stocks_df.*, value_sorts.value_portfolio, value_sorts.value_q30, value_sorts.value_q70
        FROM main_stocks_df
        LEFT JOIN value_sorts
        ON main_stocks_df.reference_date=value_sorts.reference_date AND main_stocks_df.ticker=value_sorts.ticker
        '''

main_stocks_df_value=pd.read_sql(query, conn)

In [ ]:
main_stocks_df=main_stocks_df_value
main_stocks_df

,index,Unnamed: 0,ticker,exchange,date,ret,quarter,mkt_cap,reference_date,book_value,mkt_equity,bm_ratio,size_median,size_portfolio,value_portfolio,value_q30,value_q70
0,0,1610,AA,NYSE,2019-06-03 00:00:00,1.510146,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B,H,0.199429,0.537888
1,1,1611,AA,NYSE,2019-06-04 00:00:00,1.766620,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B,H,0.199429,0.537888
2,2,1612,AA,NYSE,2019-06-05 00:00:00,-2.375514,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B,H,0.199429,0.537888
3,3,1613,AA,NYSE,2019-06-06 00:00:00,0.935891,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B,H,0.199429,0.537888
4,4,1614,AA,NYSE,2019-06-07 00:00:00,-0.602689,2,5224.02,2019-06-01,7588.0000,7534.36,1.007119,3003.895,B,H,0.199429,0.537888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210422,2213878,3974412,ZYNE,NASDAQ,2022-12-21 00:00:00,1.929825,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S,M,0.131842,0.485510
2210423,2213879,3974413,ZYNE,NASDAQ,2022-12-22 00:00:00,-7.056799,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S,M,0.131842,0.485510
2210424,2213880,3974414,ZYNE,NASDAQ,2022-12-23 00:00:00,-0.018519,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S,M,0.131842,0.485510
2210425,2213881,3974415,ZYNE,NASDAQ,2022-12-27 00:00:00,-7.390257,4,33.49,2022-06-01,70.9133,174.91,0.405427,2822.850,S,M,0.131842,0.485510


In [ ]:
main_stocks_df.to_csv(filepath + "pre sort: cleaned returns with size and value sorts")